In [12]:
import os
import sys

In [4]:
direct='../'
exec(open('../LoadData.py').read())

In [9]:
from dpcomp_core.algorithm import *
from dpcomp_core.query_nd_union import ndRangeUnion
from dpcomp_core import workload


In [10]:
def get_hist(col, epsilon, seed):
    cnts = pd.value_counts(col)
    h_maker = identity.identity_engine()
    Q = workload.Identity((len(cnts.index),))
    cnts = h_maker.Run(Q, cnts, epsilon, seed)
    cnts /= cnts.sum()
    return cnts
    
def rand_seed(seed):
    prng = np.random.RandomState(seed)
    while(True):
        yield prng.randint(0, 500000)
        
def DPNaiveBayes(db, budget, seed=12345, ret_preds=False):
    k = len(db.x_names)
    epsilon = budget/(k+1)
    ys = db.train[db.y_name]
    ycat = db.train[db.y_name].cat.categories
    rand_int = rand_seed(seed)
    
    Hy = get_hist(db.train[db.y_name], epsilon, next(rand_int))
    Hxs = dict([(c, dict([(y, get_hist(db.train[ys==y][c], epsilon, next(rand_int))) for y in ycat])) for c in db.x_names])
    probs = np.array([np.array([Hxs[c][y][db.test[c]] for c in db.x_names]).prod(axis=0)*Hy[y] for y in ycat]).T
    preds = probs.argmax(axis=1)
    acc = (preds == db.test[db.y_name]).sum() / len(db.test)
    if(ret_preds):
        return acc, preds
    else:
        return acc
    

In [1]:
def get_hist(col, epsilon, seed):
    cnts = pd.value_counts(col)
    cnts /= epsilon
    Q = workload.Identity((len(cnts.index),))
    D = Data(cnts, len(col.cat.categories), len(col), False, 1.0)
    cnts = cm.mkChoice(D)
    cnts /= cnts.sum()
    return cnts

In [11]:
DPNaiveBayes(nurs, 20, 1245)

0.907664609053498

In [177]:
A = algorithm.identity.identity_engine()

In [178]:
col = nurs.train[nurs.y_name]
cnts = pd.value_counts(col)
cnts

0    3017
1    3001
3    2812
4     241
2       1
Name: 8, dtype: int64

In [174]:
Q = []
for c in cnts.index:
    q = ndRangeUnion()
    q.add1DRange(c, c)
    Q.append(q)
W = workload.Workload(Q, len(cnts.index))
cnts = A.Run(Q, cnts, 5, 12345)
cnts /= cnts.sum()

In [175]:
cnts

0    0.332607
1    0.330790
3    0.309944
4    0.026546
2    0.000113
Name: 8, dtype: float64

In [153]:
A.Run(None, cnts, 3, 134)

0    3017.325000
1    3000.963220
3    2812.098344
4     241.481022
2       1.133567
Name: 8, dtype: float64

In [4]:
exec(open('Algorithms.py').read())
exec(open('ChoiceMaker.py').read())

In [ ]:
alg_list = [Dawa(), Identity(), Hb()]
mfs = HistMetafeats()
d = DTree(3, group_gini)
cm = ChoiceMaker.create(train_set, alg_list, d, mfs)